In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_black
%load_ext nb_mypy

In [ ]:
import nbimporter
from Exercise03AI import Exercise03AI

# Aufgabe 04: Minimax (Simplified Evaluation Function) mit Memoisierung

Dieses Notebook implementiert die Memoisierungsfunktion für den Minimax-Algorithmus zur Berechnung des nächsten Zuges im Mittelspiel. Hierbei wird die `minimax`-Funktion der Super-Klasse (`Exercise03AI`) aufgerufen und durch die nun implementierte Memoisierung beschleunigt.

Wie auch schon in `Exercise03AI` existiert eine Funktion `reset`, die dazu dient, die Instanz nach einem beendeten Spiel in ihren Ursprungszustand zurückzusetzen. Diese hat dieselbe Funktion wie die der Elternklasse, leert zusätzlich aber noch den implementierten Cache.

In [ ]:
import chess
from typing import Any


class Exercise04AI(Exercise03AI):
    """Chooses middle game moves using minimax algorithm and piece square tables."""

    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        self.cache: dict[tuple, tuple[int, chess.Move]] = {}

    def reset(self) -> None:
        """Resets all internal variables."""
        super().reset()
        self.cache.clear()

## Memoisierung

Wie auch in anderen Spielen kann im Schach eine Position mehrfach auftreten. Die Sequenz von Zügen welche zu einer solchen Position führt, wird Transposition genannt [3]. In diesem Fall gibt es eine andere Sequenz welche zu derselben Position führt. Hieraus folgt, dass es weniger eindeutige Positionen als verschiedene Spielsequenzen gibt.  Dieser Umstand kann nun genutzt werden, um die Anzahl der zu untersuchenden Knoten im Suchbaum durch die sog. Memoisierung zu verringern [2]. Hierbei wird das Ergebnis von jedem berechneten Knoten mit der ausgehenden Position und der Suchtiefe als Identifikator in einem Cache gespeichert. Wenn bei der Berechnung nun dieselbe Position erneut auftritt und die Suchtiefe ebenfalls übereinstimmt, kann das Ergebnis aus dem Cache verwendet werden und wird nicht neu berechnet. Dieses Vorgehen beschleunigt die Suche insgesamt, da nun weniger Knoten untersucht werden müssen, führt aber auch zu einem höheren Speicherverbrauch.

## Memoisierungsfunktion

Die Klasse wird um eine Variable `cache` erweitert. Initial ist `cache` ein leeres Dictionary, welches als Schlüssel ein Tupel annimmt und als Wert ein beliebiges Objekt zurückgibt.

Die Funktion `minimax` wird überschrieben und stellt eine memoisierte Version der Minimax-Implementierung in `Exercise03AI` dar. 

Die memoisierte `minimax` Funktion versucht zunächst, den Rückgabewert der Ursprungsfunktion aus dem `cache` auszulesen. Dabei werden als Schlüssel die  Funktionsargumente zusammen mit weiteren Attributen verwendet. Beinhaltet der `cache` ein Ergebnis für die gegebenen Argumente `args`, so wird dieses zurückgegeben. Andernfalls wird die `minimax` Funktion der Elternklasse aufgerufen, der Rückgabewert wird für die verwendeten Argumente im `cache` hinterlegt und schlussendlich wird der berechnete Wert zurückgegeben.  
Der Cache wird bei jedem Bauernzug, Schlagen einer Figur und veränderten Rochade- oder En Passant-Rechten zurückgesetzt, da in diesem Fall die zuvor berechneten Stellungen nicht wieder auftreten können.

### Cache-Schlüssel

Eine wichtige Rolle spielt der Key, welcher für die Speicherung bereits berechneter Ergebnisse im `cache`-Dictionary verwendet wird. Um den Cache möglichst effektiv nutzen zu können, sollten nur die Attribute in die Berechnung des Schlüssels miteinbezogen werden, welche auch durch den Minimax-Algorithmus verwendet werden und sich im Laufe des Spieles verändern. Die Funktion `get_key` berechnet den Cache-Schlüssel und gibt ein Tupel mit den folgenden Elementen zurück:

- Eine Repräsentation des aktuellen Boards mit folgenden Informationen. Eine Bitmap für jede auf dem Spielfeld platzierten Figuren pro Figurentyp sowie eine allgemeine Bitmap aller durch die Farbe unterschiedenen besetzten Felder, einen Boolean für den aktuellen Spieler und eine Bitmap für die Rochaderechte.
- Der Zähler der Halbzüge seit dem letzten Bauernzug oder Schlagen einer Figur. Dieses Attribut wird benötigt um ein unbeabsichtigtes Remis durch die 50-Züge-Regel (Fifty-move rule) zu verhindern. Aus Effizienz-Gründen wird der Zähler nur dann inkludiert, wenn er für die Berechnung relevant ist, das heißt ein solches Remis mit der aktuellen Tiefe auftreten kann. Ansonsten wird der statische Wert `-42` verwendet.
- Die aktuelle Suchtiefe der Evaluierung.

### Seiteneffekte

In einigen Fällen kann die hier genutze Memoisierung des Minimax zu einer anderen Evaluierung und damit zu einer anderen Zugentscheidung führen. Dies ist dem Umstand geschuldet, dass ein Ergebnis aus dem Cache mit einer älteren Transpositionstabelle berechnet wurde und eine der ausgewerteten Stellungen nun mittlerweile möglicherweise als Remis gewertet werden müsste. Im schlimmsten Fall kann dieser Umstand zu einem unbeabsichtigten Remis führen, im besten Fall führt er nur zu einer leicht anderen Zugfolge mit demselben Spielausgang.
Eine allgemeine Lösung für diese Problem besteht darin, die gesamte Transpositionstabelle als Bestandteil des Cache-Schlüssels zu verwenden. Dies führt aber dazu, dass der Cache dann nur sehr selten verwendet werden kann und die Geschwindigkeitsvorteile der Memoisierung nahezu vollständig verloren gehen. Diese Probleme bei der Memoisierung werden in [4] treffend zusammengefasst:
> "[The] Use of a transposition table can lead to incorrect results if the graph-history interaction problem is not studiously avoided. This problem arises in certain games because the history of a position may be important. [...] Another example [in chess] is draw by repetition: given a position, it may not be possible to determine whether it has already occurred. A solution to the general problem is to store history information in each node of the transposition table, but this is inefficient and rarely done in practice."[4]

Eine weitere Diskussion des Problems findet sich in [5]. In dieser Implementierung wird das Problem bewusst in Kauf genommen, um die Suchgeschwindigkeit zu erhöhen. In der Praxis tritt dieses Problem hauptsächlich gegen andere KI-Versionen auf, da ein Mensch eine Zugwiederholung von Natur aus eher vermeidet.

In [ ]:
from typing import Any
import sys


class Exercise04AI(Exercise04AI):  # type: ignore
    @staticmethod
    def get_key(board: chess.Board, depth: int) -> tuple:
        """Calculates a key that uniquely identifies a given board."""
        return (
            board.pawns,
            board.knights,
            board.bishops,
            board.rooks,
            board.queens,
            board.kings,
            board.occupied_co[chess.WHITE],
            board.occupied_co[chess.BLACK],
            board.turn,
            board.castling_rights,
            board.halfmove_clock if board.halfmove_clock >= (50 - depth) else -42,
            depth,
        )

    def minimax(self, board: chess.Board, depth: int, current_evaluation: int) -> int:
        """Memoized version of the Exercise03AI minimax implementation."""
        key = Exercise04AI.get_key(board, depth)
        self.stats[-1]["cache_tries"] += 1
        if key in self.cache:
            self.stats[-1]["cache_hits"] += 1
            return self.cache[key]
        result = super().minimax(board, depth, current_evaluation)
        self.cache[key] = result
        return result

    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move:
        """Gets the best next move."""
        self.stats[-1]["cache_tries"] = 0
        self.stats[-1]["cache_hits"] = 0
        next_move = super().get_next_middle_game_move(board)
        if board.is_irreversible(next_move):
            self.cache.clear()
            self.stats[-1]["cache_cleared"] = True
        else:
            self.stats[-1]["cache_cleared"] = False
        self.stats[-1]["cache_size_mb"] = round(
            sys.getsizeof(self.cache) / (1024 * 1024), 2
        )
        return next_move

## Debugging Bereich

Die folgenden Zellen enthalten Unit-Tests der oben implementierten Funktionen.

In [ ]:
from AIBaseClass import ChessAI
import Exercise02AI as Exercise02AI_
import Exercise03AI as Exercise03AI_

In [ ]:
# Create player and board
unit_test_player = Exercise04AI(player_name="Ex04AI", search_depth=3)
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
board

Im Folgenden wird die Funktion `get_key` getestet, welche einen definierten Schlüssel für den Cache anhand des aktuellen Boards zurückgibt. Hierbei werden der `depth` Parameter als auch der Rückgabewert für die Anzahl der Halbzüge überprüft, welche, sofern die Anzahl der Halbzüge weniger als 50 entspricht, $-42$ entspricht.

In [ ]:
# Test key generation
def test_key_generation(unit_test_player: ChessAI, board: chess.Board):
    key = Exercise04AI.get_key(board, depth=42)
    (half_moves_key, depth) = key[-2:]
    print(f"Half moves key: {half_moves_key}")
    print(f"Depth: {depth}")
    assert half_moves_key == -42, "Half moves key does not match expected value!"
    assert depth == 42, "Depth does not match expected value!"

In [ ]:
test_key_generation(unit_test_player, board)

Da der Minimax im Vergleich zur `Exercise02AI` durch die Memoisierung erweitert wurde, wird nun auch der Unittest erweitert. Es werden die zusätzlichen Metriken `cache_tries` und `cache_hits` überprüft. Auf das gegebe Board wird nun der Zug angewendet, welcher die weiße Dame schlägt. Dadurch beträgt die erwartete Evaluierung $4$. Die Anzahl der Nodes sowie die Cache Metriken dienen dazu, Änderungen in der Komplexität der Berechnung aufzuzeigen.

In [ ]:
# Test minimax
def test_minimax(
    unit_test_player: ChessAI,
    board: chess.Board,
    current_evaluation: int,
    expected_evaluation: int,
    expected_move: str,
    expected_nodes: int,
    expected_cache_tries: int,
    expected_cache_hits: int,
    expected_cache_elements: int,
):
    unit_test_player.cache = {}  # Clear cache
    unit_test_player.stats[-1]["cache_tries"] = 0
    unit_test_player.stats[-1]["cache_hits"] = 0
    Exercise02AI_.test_minimax(
        unit_test_player,
        board,
        current_evaluation,
        expected_evaluation,
        expected_move,
        expected_nodes,
    )
    cache_tries = unit_test_player.stats[-1]['cache_tries']
    cache_hits = unit_test_player.stats[-1]['cache_hits']
    print(f"Cache tries: {cache_tries}")
    print(f"Cache hits: {cache_hits}")
    print(f"Elements in cache: {len(unit_test_player.cache)}")
    assert cache_tries == expected_cache_tries, "Cache tries do not match expected value!"
    assert cache_hits == expected_cache_hits, "Cache hits do not match expected value!"
    assert len(unit_test_player.cache) == expected_cache_elements, "Cache elements do not match expected value!"


In [ ]:
test_minimax(
    unit_test_player,
    board,
    current_evaluation=1240,
    expected_evaluation=325,
    expected_move="d4c3",
    expected_nodes=8651,
    expected_cache_tries=14377,
    expected_cache_hits=5726,
    expected_cache_elements=8651,
)

Um zu validieren, dass die Memoisierung das gewünschte Ergebnis zurückliefert, wird die Minimax Testfuntion der `Exercise02AI` erneut mit den selben Evaluierungsparametern aufgerufen, erwartet werden allerdings andere Metriken. Das Ergebnis der Funktion soll dasselbe sein, es soll jedoch nur einmal auf den Cache zugegriffen werden, wodurch direkt das richtige Ergebnis zurückgeliefert werden soll.

In [ ]:
# Test minimax again (now memoized)
def test_memoized_minimax(
    unit_test_player: ChessAI,
    board: chess.Board,
    current_evaluation: int,
    expected_evaluation: int,
    expected_move: str,
    expected_nodes: int,
    expected_cache_tries: int,
    expected_cache_hits: int,
    expected_cache_elements: int,
):
    unit_test_player.stats[-1]["cache_tries"] = 0
    unit_test_player.stats[-1]["cache_hits"] = 0
    Exercise02AI_.test_minimax(
        unit_test_player,
        board,
        current_evaluation,
        expected_evaluation,
        expected_move,
        expected_nodes,
    )
    cache_tries = unit_test_player.stats[-1]["cache_tries"]
    cache_hits = unit_test_player.stats[-1]["cache_hits"]
    print(f"Cache tries: {cache_tries}")
    print(f"Cache hits: {cache_hits}")
    print(f"Elements in cache: {len(unit_test_player.cache)}")
    assert (
        cache_tries == expected_cache_tries
    ), "Cache tries do not match expected value!"
    assert cache_hits == expected_cache_hits, "Cache hits do not match expected value!"
    assert (
        len(unit_test_player.cache) == expected_cache_elements
    ), "Cache elements do not match expected value!"

In [ ]:
test_memoized_minimax(
    unit_test_player,
    board,
    current_evaluation=1240,
    expected_evaluation=325,
    expected_move="d4c3",
    expected_nodes=0,
    expected_cache_tries=1,
    expected_cache_hits=1,
    expected_cache_elements=8651,
)

Um die Funktion `get_next_middle_game_move` zu testen, wird überprüft, ob diese korrekterweise den Zug zurückgibt, in welchem die Dame durch den schwarzen Bauern geschlagen wird. Durch die implementierte Memoisierung sollen hierbei keine neuen Knoten berechnet werden müssen.

In [ ]:
# Test next move function (with memoized minimax result)
Exercise02AI_.test_next_move(
    unit_test_player,
    board,
    expected_move="d4c3",
    expected_nodes=0,
)

Die `reset` Funktion wurde um die Funktionalität erweitert, den implementierten Cache zu bereinigen. Um zu validieren, dass das Zurücksetzen wie erwartet funktioniert, wird die Testfunktion der Elternklasse aufgerufen, welche die bisherigen Tests überprüft. Zusätzlich wird überprüft, ob der Cache nach dem Zurücksetzen einem leeren Dictionary entspricht.

In [ ]:
# Test reset function
def test_reset(unit_test_player: ChessAI, board: chess.Board):
    Exercise03AI_.test_reset(unit_test_player, board)
    assert unit_test_player.cache == {}, "Reset was not successful! (cache)"

In [ ]:
test_reset(unit_test_player, board)

## Temporärer Bereich

Der folgende Bereich dient zum temporären Debuggen und kann nicht-funktionierenden Code enthalten. Dieser Bereich wird vor der Abgabe entfernt.